In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 1. Load the dataset
csv_path = 'final_workspace_volumes.csv'  # adjust path as needed
df = pd.read_csv(csv_path)

# 2. One-Hot Encode the shape columns
def one_hot_encode_shape(shape_column):
    """Converts a string like '[0, 1, 0]' into three separate columns."""
    return pd.DataFrame(shape_column.apply(eval).to_list(),
                        columns=['Cuboid', 'Sphere', 'Cylinder'])

# encode Object 1 and Object 2 shapes
df_obj1 = one_hot_encode_shape(df['Object 1 Shape (Cuboid, Sphere, Cylinder)'])
df_obj2 = one_hot_encode_shape(df['Object 2 Shape (Cuboid, Sphere, Cylinder)'])
df = pd.concat([df, df_obj1.add_prefix('Obj1_'), df_obj2.add_prefix('Obj2_')], axis=1)
df.drop(columns=['Object 1 Shape (Cuboid, Sphere, Cylinder)',
                 'Object 2 Shape (Cuboid, Sphere, Cylinder)'],
        inplace=True)

# 3. Define features (X) and targets (y)
X = df.drop(columns=[
    'Remaining Workspace Volume after first grasp',
    'Remaining Workspace Volume after both grasps'
])
y = df[[
    'Remaining Workspace Volume after first grasp',
    'Remaining Workspace Volume after both grasps'
]]

# 4. Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 5. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [2]:
baseline_constructors = {
    'LinearRegression': LinearRegression,
    'DecisionTree':    lambda: DecisionTreeRegressor(random_state=42),
    'KNeighbors':      KNeighborsRegressor,
    'DummyMean':       lambda: DummyRegressor(strategy='mean')
}

# 7. Train, predict, and evaluate
results = []

In [3]:



for name, Constructor in baseline_constructors.items():
    # instantiate two separate models
    m1 = Constructor()
    m2 = Constructor()
    
    # fit each on its respective target
    m1.fit(X_train, y_train.iloc[:, 0])    # first-grasp volume
    m2.fit(X_train, y_train.iloc[:, 1])    # second-grasp volume
    
    # predict
    p1 = m1.predict(X_test)
    p2 = m2.predict(X_test)
    
    # evaluate
    mae1 = mean_absolute_error(y_test.iloc[:, 0], p1)
    mae2 = mean_absolute_error(y_test.iloc[:, 1], p2)
    mse1 = mean_squared_error(y_test.iloc[:, 0], p1)
    mse2 = mean_squared_error(y_test.iloc[:, 1], p2)
    r2  = r2_score(
        y_test,
        pd.DataFrame({'first': p1, 'second': p2}),
        multioutput='variance_weighted'
    )
    
    results.append({
        'Model': name,
        'MAE (1st grasp)': mae1,
        'MAE (2nd grasp)': mae2,
        'MSE (1st grasp)': mse1,
        'MSE (2nd grasp)': mse2,
        'R² overall':      r2
    })


In [4]:
results_df = pd.DataFrame(results).set_index('Model')
print(results_df)

                  MAE (1st grasp)  MAE (2nd grasp)  MSE (1st grasp)  \
Model                                                                 
LinearRegression         0.000008         0.000006     1.253625e-10   
DecisionTree             0.000004         0.000001     1.200194e-10   
KNeighbors               0.000010         0.000002     1.815659e-10   
DummyMean                0.000034         0.000012     2.116145e-09   

                  MSE (2nd grasp)  R² overall  
Model                                          
LinearRegression     7.872129e-11    0.919348  
DecisionTree         6.860383e-12    0.949858  
KNeighbors           1.690784e-11    0.921565  
DummyMean            4.200341e-10   -0.002278  
